### Libraries

In [1]:
import sys
sys.path.insert(0,'/Users/jarad/Fake Folder/Python Libraries')

from jb_libraries import *
%matplotlib inline

### Script settings

In [2]:
csv_path = '/Users/jarad/Fake Folder/Fab/Recurring/Time to Test and Prep Fab Parts/CSVs/'
close_workbook = 'yes'

In [3]:
n = 90
now = dt.datetime.now().date()
x_days_ago = (now - pd.DateOffset(days = n)).date()

date_start = str(x_days_ago)
date_end = str(now)

print('Data is from {} to {}, which is {} days'.format(date_start,date_end,n))

Data is from 2019-02-12 to 2019-05-13, which is 90 days


In [4]:
workbook_title = 'Previous 90 days ending {} - Time to Test and Prep Fab Parts'.format(date_end)

### Get data

In [5]:
processes_main = pd.read_sql(
'''
SELECT
process_id,
wo_id,
process_group
FROM processes
WHERE DATE(time_start) BETWEEN ' '''+ date_start +''' ' AND ' '''+ date_end +''' ' 
AND wo_id != 0 # to exclude reworks
AND process_group IN ('test','prep','secondary') # get only these groups
AND time_end != '0000-00-00 00:00:00' # to exclude open processes
''', db)

col_fix(processes_main)

In [6]:
p = list(set(processes_main['process id']))

sp_actions_main = pd.read_sql(
'''
SELECT
process_id,
spa_qty,
spa_time_start,
spa_time_end
FROM sp_actions
WHERE process_id IN '''+ str(tuple(p)) +'''
''', db)

col_fix(sp_actions_main)

e = sp_actions_main['spa time end'] - sp_actions_main['spa time start']
minutes = (e.dt.days * 24 * 60 * 60 + e.dt.seconds)/60
sp_actions_main['time elapsed in minutes'] = minutes

In [7]:
skus_main = pd.read_sql(
'''
SELECT
sku_id,
k.part_id,
pd.products_name
FROM skus k
LEFT JOIN products_description pd ON k.part_id = pd.part_id
''', db)

col_fix(skus_main)

In [8]:
w = list(set(processes_main['wo id']))

work_orders_main = pd.read_sql(
'''
SELECT
wo_id,
sku_id,
wo_qty AS work_order_qty
FROM work_orders 
WHERE wo_id IN '''+ str(tuple(w)) +'''
AND wo_status = 'completed' # only include completed WOs
''', db)

col_fix(work_orders_main)

for c in ['part id','products name']:
    work_orders_main[c] = work_orders_main['sku id'].map(dict(zip(skus_main['sku id'], skus_main[c])))
    
work_orders_main['products name'].fillna('no part name', inplace = True)    

### Structure it

In [9]:
main = sp_actions_main.copy()

# map from processes_main
cols = processes_main.columns.tolist()
cols.remove('process id')
for c in cols:
    main[c] = main['process id'].map(dict(zip(processes_main['process id'],processes_main[c])))
    
# map from work_orders_main    
cols = work_orders_main.columns.tolist()
cols.remove('wo id')
for c in cols:
    main[c] = main['wo id'].map(dict(zip(work_orders_main['wo id'],work_orders_main[c])))
    
# check nulls    
n = main[main.isnull().any(1)]
if n.empty == True: # if there are no nulls, move on
    pass
else: # if there are nulls, check the wo_status of these wo_ids; if the statuses are anything but "completed", move on
    
    n_ls = list(set(n['wo id']))

    check = pd.read_sql(
    '''
    SELECT
    wo_status,
    COUNT(*) AS count
    FROM work_orders
    WHERE wo_id IN '''+ str(tuple(n_ls)) +'''
    ''', db)

    col_fix(check)    

    count = 0
    for index, row in check.iterrows():
        count += row['count']
        if row['wo status'] == 'completed':
            raise ValueError('check your nulls')

            if count != len(n_ls): # if the count of nulls from your df matches that in the db, move on
                raise ValueError('check your nulls')

main.dropna(inplace = True)
main.reset_index(drop = True, inplace = True)

### Get time elapsed

In [10]:
time = main.groupby('part id')[['time elapsed in minutes']].sum()

### Get WO qty

In [11]:
wo_qty = work_orders_main.groupby('part id')[['work order qty']].sum()

### Get Wo Ids

In [12]:
a1 = work_orders_main[['part id','wo id']].drop_duplicates()
a2 = a1.groupby('part id')['wo id'].apply(lambda x: ', '.join(x.map(str)))
wo_ids = pd.DataFrame(a2)
wo_ids.columns = ['wo ids']

### Data by part

In [13]:
by_part = time.join(wo_qty).join(wo_ids)
by_part.reset_index(inplace = True)
by_part['part id'] = by_part['part id'].map(int)

by_part['products name'] = by_part['part id'].map(dict(zip(main['part id'], main['products name'])))
    
by_part['min per'] = by_part['time elapsed in minutes']/by_part['work order qty']

l1 = 'https://volcano.adafruit.com/volcano/ada_plm.php?/Parts/'
l2 = '/PartProfile/'

by_part['link'] = l1 + by_part['part id'].map(str) + l2

### Rearrange columns and sort

In [14]:
new_cols = ['part id',
            'products name',
            'wo ids',
            'time elapsed in minutes',
            'work order qty',
            'min per',
            'link']

old_cols = by_part.columns.tolist()

s1 = set(old_cols)
s2 = set(new_cols)
s3 = s1.symmetric_difference(s2)

if len(s3) != 0:
    raise ValueError('check ur columns')
else:
    by_part = by_part[new_cols]
    
by_part.sort_values('time elapsed in minutes', ascending = False, inplace = True)    

In [15]:
by_part.head()

,part id,products name,wo ids,time elapsed in minutes,work order qty,min per,link
147,4116,Adafruit PyPortal - CircuitPython Powered Internet Display,"8288, 8333, 8350, 8391, 8401, 8429, 8432, 8479, 8511",5247.966667,4868,1.078054,https://volcano.adafruit.com/volcano/ada_plm.php?/Parts/4116/PartProfile/
65,2598,Adafruit Feather M0 WiFi - ATSAMD21 + ATWINC1500 (accounts only),8227,3275.316667,1120,2.924390,https://volcano.adafruit.com/volcano/ada_plm.php?/Parts/2598/PartProfile/
112,3382,Adafruit Metro M4 feat. Microchip ATSAMD51,"8003, 8342",3229.650000,981,3.292202,https://volcano.adafruit.com/volcano/ada_plm.php?/Parts/3382/PartProfile/
95,3178,Adafruit Feather M0 with RFM95 LoRa Radio - 900MHz,"7994, 8281",1875.450000,1669,1.123697,https://volcano.adafruit.com/volcano/ada_plm.php?/Parts/3178/PartProfile/
64,2590,Adafruit Metro Mini 328 - Arduino-Compatible - 5V 16MHz,7702,1708.466667,1800,0.949148,https://volcano.adafruit.com/volcano/ada_plm.php?/Parts/2590/PartProfile/


### Find outliers by time elapsed

In [16]:
for_outliers = main.groupby('process group').agg({'time elapsed in minutes':['mean','std']})
for_outliers.columns = for_outliers.columns.droplevel(0)
for_outliers['upper'] = for_outliers['mean'] + 3 * for_outliers['std']
for_outliers.reset_index(inplace = True)

def find_outlier(df):
    
    group = df['process group']
    time = df['time elapsed in minutes']
    upper = for_outliers[for_outliers['process group'] == group]['upper'].item()
    
    if time > upper:
        return 'yes'
    else:
        return 'no'
    
main['outlier'] = main.apply(find_outlier, axis = 1)

In [17]:
print('outliers below')
main[main['outlier'] == 'yes'].sort_values(['process group','time elapsed in minutes'], ascending = [False,False])

outliers below


,process id,spa qty,spa time start,spa time end,time elapsed in minutes,wo id,process group,sku id,work order qty,part id,products name,outlier
1266,176904,511,2019-03-07 07:52:43,2019-03-07 13:21:39,328.933333,8350,test,13180.0,1000.0,4116.0,Adafruit PyPortal - CircuitPython Powered Internet Display,yes
1405,178064,192,2019-03-26 08:17:08,2019-03-26 13:36:43,319.583333,8418,test,13331.0,192.0,4172.0,Adafruit HUZZAH32 â ESP32 Breakout Board,yes
1365,177561,962,2019-03-11 11:05:23,2019-03-11 15:55:48,290.416667,8391,test,13180.0,1000.0,4116.0,Adafruit PyPortal - CircuitPython Powered Internet Display,yes
1312,177464,21,2019-03-26 09:24:13,2019-03-26 14:09:39,285.433333,8382,test,13036.0,240.0,4075.0,Adafruit LoRa Radio Bonnet RFM96W @ 433MHz,yes
1411,178379,225,2019-03-14 10:56:50,2019-03-14 15:29:46,272.933333,8429,test,13180.0,1184.0,4116.0,Adafruit PyPortal - CircuitPython Powered Internet Display,yes
1414,178379,480,2019-03-15 09:30:14,2019-03-15 13:56:51,266.616667,8429,test,13180.0,1184.0,4116.0,Adafruit PyPortal - CircuitPython Powered Internet Display,yes
1388,177810,38,2019-03-14 09:50:04,2019-03-14 14:12:35,262.516667,8401,test,12921.0,200.0,4116.0,Adafruit PyPortal - CircuitPython Powered Internet Display,yes
40,159345,234,2019-03-20 08:16:49,2019-03-20 12:26:28,249.650000,7702,test,8817.0,1800.0,2590.0,Adafruit Metro Mini 328 - Arduino-Compatible - 5V 16MHz,yes
1479,179381,70,2019-05-06 11:37:09,2019-05-06 15:43:04,245.916667,8479,test,12921.0,136.0,4116.0,Adafruit PyPortal - CircuitPython Powered Internet Display,yes
1247,176681,120,2019-04-26 08:07:55,2019-04-26 12:12:32,244.616667,8342,test,12652.0,186.0,3382.0,Adafruit Metro M4 feat. Microchip ATSAMD51,yes


### Check it

In [18]:
# check nulls
n = by_part[by_part.isnull().any(1)]
if n.empty == False:
    raise ValueError('check ur nulls')
    
# qty totals    
t1 = work_orders_main['work order qty'].sum()
t2 = by_part['work order qty'].sum()

if t1 - t2 != 0:
    raise ValueError('check ur totals')    
    
# part count
s1 = set(work_orders_main['part id'])
s2 = set(by_part['part id'])

if len(s1) != len(s2):
    raise ValueError('check ur part count')

*****
# Excel
*****

### Set up workbook

In [19]:
workbook = xlsxwriter.Workbook(csv_path + workbook_title + '.xlsx',
                               {'nan_inf_to_errors': True,
                               'strings_to_numbers': True})

tabs = ['Data By Part']

#=========================
# Formats
#=========================

colors = ['#343635',
          '#2e4874',
          '#7eaba4',
          '#928c85',
          '#347c83',
          '#bfb9d6']
colors = colors * 2

title = workbook.add_format({'font_size':25,
                             'font_name':'Arial (Bold)'})

subtitle = workbook.add_format({'font_size':15,
                                'font_name':'Arial (Bold)'})

col_names = workbook.add_format({'font_name':'Arial (Bold)',
                                 'font_color':'white',
                                 'valign':'vcenter',
                                 'align':'center',
                                 'bg_color':colors[2],
                                 'bottom':1,
                                 'top':1,
                                 'left':1,
                                 'right':1})

right_align = workbook.add_format({'align':'right'})

number = workbook.add_format({'num_format':'#,##0'})
number2 = workbook.add_format({'num_format':'#,##0.0'})
dummy = workbook.add_format({'font_color':'black'})

for tab in tabs:
    workbook.add_worksheet(tab) # create each tab

my_worksheets = {}
for sht in workbook.worksheets():
    my_worksheets[sht.get_name()] = sht # create dict like tab_name:worksheet_instance

#=========================
# Apply same formatting to each worksheet
#=========================

for k, v in my_worksheets.items():
    sht = my_worksheets[k]
    sht.write(0, 0,
              k.title(),
              title)
    sht.write(1, 0,
              workbook_title,
              subtitle)

### Write worksheets

In [20]:
sht = my_worksheets['Data By Part']

fmt = {'time elapsed in minutes':number,
      'work order qty':number,
      'min per':number2,
      'wo ids':right_align}

df = by_part.copy()

start_row = 3
start_col = 0

#=========================
# disclaimer 01
#=========================

ls = list(set(main['process group']))
w = '*times are summed over the '
for i in range(len(ls)):
    if i != len(ls) - 1:
        w += ls[i] + ', '
    else:
        w += 'and ' + ls[i] + ' steps, and the data only includes completed work orders.'
        
sht.write(start_row - 1,
         start_col,
         w)

#=========================
# data
#=========================

for i in range(len(df.columns)):
    sht.write(start_row + 1,
              start_col + i,
              df.columns[i].title(),
              col_names)

    len1 = [len(str(x)) for x in df.iloc[:, i]]
    len2 = [len(df.columns[i])]
    len3 = np.max(len1 + len2)

    sht.set_column(start_col + i,
                   start_col + i,
                   len3)

    for j in range(len(df)):
        sht.write(start_row + 2 + j, 
                  start_col + i,
                  df.iloc[j,i],
                  fmt[df.columns[i]] if df.columns[i] in list(fmt.keys()) else dummy)
        
if close_workbook == 'yes':
    workbook.close()        

In [21]:
print('done')

done
